# SMA Pullback Trend-Following (1D/4H/1H)

Hypothesis: in a strong uptrend (Close > SMA200 and SMA50 > SMA200), the optimal swing entry appears after a pullback to EMA20 and renewed momentum.


## Indicators
- SMA200 (long-term trend)
- SMA50 (mid-term trend)
- EMA20 (pullback detection)
- RSI(14) momentum filter
- Volume confirmation (optional)

Exits are managed by ManagedStrategy risk controls (SL / TP / trailing).


In [8]:
from pathlib import Path
import sys

def find_project_root(start=None):
    start = Path.cwd() if start is None else Path(start)
    for candidate in [start] + list(start.parents):
        if (candidate / 'utils').exists() and (candidate / 'backtesting').exists():
            return candidate
    return start

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('Project root:', PROJECT_ROOT)


Project root: c:\Users\saill\Desktop\t_project


In [9]:
# Notebook configuration (edit as needed)
TICKER = 'SPY'
START_DATE = '2015-01-01'
END_DATE = '2025-01-01'

TIMEFRAME_DAILY = '1d'
TIMEFRAME_INTRADAY = '1h'
RESAMPLE_TO = '4H'  # resample intraday feed to this timeframe

RISK_PCT = 0.02  # 2% risk per trade
DEFAULT_STOP_DISTANCE = 0.03  # used by FixedFractionalSizer


In [10]:
import pandas as pd

from utils.data_manager import DataManager
from utils.data_processor import resample_data

dm = DataManager()
df_daily = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_DAILY)
df_intraday = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_INTRADAY)

if df_intraday is not None and not df_intraday.empty:
    df_4h = resample_data(df_intraday, rule=RESAMPLE_TO)
else:
    df_4h = pd.DataFrame()
    print('Intraday data unavailable; continuing without 4H resample.')

print(f'Daily bars: {len(df_daily)}')
if not df_4h.empty:
    print(f'4H bars: {len(df_4h)}')


2025-11-03 20:58:23 - utils.data_manager - INFO - DataManager initialisé. Cache: C:\Users\saill\Desktop\t_project\data\cache. Timezone: Europe/Paris
2025-11-03 20:58:23 - utils.data_manager - INFO - Cache insuffisant pour SPY (demandé: 2015-01-01 à 2025-01-01). Re-téléchargement.
2025-11-03 20:58:23 - utils.data_manager - INFO - Téléchargement pour SPY (plage par défaut : 2015-01-01 à 2025-11-03)...
2025-11-03 20:58:23 - utils.data_manager - INFO - Téléchargement de SPY (2015-01-01 à 2025-11-03, 1d)...
2025-11-03 20:58:24 - utils.data_manager - INFO - Données téléchargées avec succès pour SPY (2725 lignes).
2025-11-03 20:58:24 - utils.data_manager - INFO - [OK] Données prêtes pour SPY (2516 lignes de 2015-01-01 à 2025-01-01).
2025-11-03 20:58:24 - utils.data_manager - WARNING - Erreur au chargement du cache C:\Users\saill\Desktop\t_project\data\cache\SPY_1h.csv: 'Date' is not in list. Re-téléchargement.
2025-11-03 20:58:24 - utils.data_manager - INFO - Téléchargement pour SPY (plage pa

Daily bars: 2516
4H bars: 866


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_plot = df_daily.copy()
try:
    import pandas_ta as ta
    df_plot['EMA20'] = ta.ema(df_plot['close'], length=20)
    df_plot['SMA50'] = ta.sma(df_plot['close'], length=50)
    df_plot['SMA200'] = ta.sma(df_plot['close'], length=200)
    df_plot['RSI14'] = ta.rsi(df_plot['close'], length=14)
except Exception:
    df_plot['EMA20'] = df_plot['close'].rolling(20).mean()
    df_plot['SMA50'] = df_plot['close'].rolling(50).mean()
    df_plot['SMA200'] = df_plot['close'].rolling(200).mean()
    # Fallback RSI placeholder
    df_plot['RSI14'] = 50.0

# Normalize timestamps for Plotly
if getattr(df_plot.index, 'tz', None) is not None:
    xs = df_plot.index.tz_convert('UTC').tz_localize(None)
else:
    xs = df_plot.index

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
fig.add_trace(go.Candlestick(x=xs, open=df_plot['open'], high=df_plot['high'], low=df_plot['low'], close=df_plot['close'], name='OHLC'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['EMA20'], mode='lines', name='EMA20'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['SMA50'], mode='lines', name='SMA50'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['SMA200'], mode='lines', name='SMA200'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['RSI14'], mode='lines', name='RSI14', line=dict(color='purple')), row=2, col=1)
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='RSI', row=2, col=1, range=[0, 100])
fig.update_layout(title=f'{TICKER} price with trend filters', xaxis_rangeslider_visible=False, template='plotly_white')
fig.show()


In [12]:
import backtrader as bt

from backtesting.engine import BacktestEngine
from risk_management.position_sizing import FixedFractionalSizer
from strategies.implementations.sma_pullback_managed_strategy import SmaPullbackManagedStrategy
from utils.config_loader import get_settings

engine = BacktestEngine()
engine.cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')
engine.cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')

engine.add_sizer(FixedFractionalSizer, risk_pct=RISK_PCT, stop_distance=DEFAULT_STOP_DISTANCE)
engine.add_data(df_daily, name='data_1d')
if 'df_4h' in globals() and not df_4h.empty:
    engine.add_data(df_4h, name='data_4h')

engine.add_strategy(
    SmaPullbackManagedStrategy,
    trend_long_period=200,
    trend_mid_period=50,
    pullback_ma_period=20,
    rsi_period=14,
    rsi_min=50,
    vol_ma_period=20,
    require_volume_confirm=True,
)

results = engine.run()
strategy_instance = results[0]


2025-11-03 20:58:24 - backtesting.engine - INFO - Initialisation du BacktestEngine...
2025-11-03 20:58:24 - backtesting.engine - INFO - Capital initial du broker fixé à : 10,000.00
2025-11-03 20:58:24 - backtesting.engine - INFO - Commission (pourcentage) fixée à : 0.1000%
2025-11-03 20:58:24 - backtesting.engine - INFO - Slippage (pourcentage) fixé à : 0.0500%
2025-11-03 20:58:24 - backtesting.engine - INFO - Position Sizer 'FixedFractionalSizer' ajouté. Paramètres: (risk_pct=0.02, stop_distance=0.03)
2025-11-03 20:58:24 - backtesting.engine - INFO - Flux de données 'data_1d' ajouté. Période: 2015-01-02 à 2024-12-31.
2025-11-03 20:58:24 - backtesting.engine - INFO - Flux de données 'data_4h' ajouté. Période: 2023-11-06 à 2024-12-31.
2025-11-03 20:58:24 - backtesting.engine - INFO - Stratégie 'SmaPullbackManagedStrategy' ajoutée. Paramètres: (trend_long_period=200, trend_mid_period=50, pullback_ma_period=20, rsi_period=14, rsi_min=50, vol_ma_period=20, require_volume_confirm=True)
2025

In [13]:
initial_capital = get_settings().get('backtest', {}).get('initial_capital', 10000.0)
try:
    from scripts.run_backtest import print_results
    print_results(results, initial_capital, df_daily)
except Exception as exc:
    final_value = strategy_instance.broker.getvalue()
    print(f'Final portfolio value: {final_value:,.2f}')
    print(f'(Detailed metrics unavailable: {exc})')



RÉSULTATS DU BACKTEST

📊 Période: 2015-01-02 à 2024-12-31
📊 Nombre de bougies: 2516

💰 PERFORMANCE
   Capital Initial:              10,000.00 €
   Capital Final:                 8,731.69 €
   P&L:                          -1,268.31 € (-12.68%)
   Retour Total:                    -13.56%
   Retour Moyen (annuel):            -0.01%

📈 TRADES
   Nombre Total:                         6
   Trades Gagnants:                      2
   Trades Perdants:                      4
   Win Rate:                        33.33%
   Gain Moyen:                      323.58 €
   Perte Moyenne:                  -478.87 €

📉 RISQUE
   Sharpe Ratio:           -0.5961505814701227
   Max Drawdown:                    15.64%



In [14]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def _to_naive(dt):
    if dt is None:
        return None
    ts = pd.Timestamp(dt)
    if ts.tzinfo is not None:
        ts = ts.tz_convert('UTC').tz_localize(None)
    return ts

# Extract transactions (buys/sells) for markers
tx = strategy_instance.analyzers.transactions.get_analysis()
buy_times, buy_prices, sell_times, sell_prices = [], [], [], []
for k, v in (tx or {}).items():
    ts = _to_naive(k)
    for t in v:
        # t structure typically: [size, price, sid]
        size = t[0] if len(t) > 0 else None
        price = t[1] if len(t) > 1 else None
        if size is None or size == 0:
            continue
        if size > 0:
            buy_times.append(ts); buy_prices.append(price)
        elif size < 0:
            sell_times.append(ts); sell_prices.append(price)

# Equity curve
time_returns = strategy_instance.analyzers.timereturn.get_analysis()
if time_returns:
    equity_series = pd.Series(time_returns).sort_index()
    equity_curve = initial_capital * (1 + equity_series).cumprod()
else:
    equity_curve = pd.Series(dtype='float64')

# Price timeline
if getattr(df_daily.index, 'tz', None) is not None:
    price_index = df_daily.index.tz_convert('UTC').tz_localize(None)
else:
    price_index = df_daily.index

fig_bt = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
fig_bt.add_trace(go.Candlestick(x=price_index, open=df_daily['open'], high=df_daily['high'], low=df_daily['low'], close=df_daily['close'], name='OHLC'), row=1, col=1)
fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['EMA20'], mode='lines', name='EMA20'), row=1, col=1)
fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['SMA50'], mode='lines', name='SMA50'), row=1, col=1)
fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['SMA200'], mode='lines', name='SMA200'), row=1, col=1)

if buy_times:
    fig_bt.add_trace(go.Scatter(x=buy_times, y=buy_prices, mode='markers', name='Buy', marker=dict(symbol='triangle-up', color='green', size=10)), row=1, col=1)
if sell_times:
    fig_bt.add_trace(go.Scatter(x=sell_times, y=sell_prices, mode='markers', name='Sell', marker=dict(symbol='triangle-down', color='red', size=10)), row=1, col=1)

if not equity_curve.empty:
    eq_index = equity_curve.index
    if getattr(eq_index, 'tz', None) is not None:
        eq_index = eq_index.tz_convert('UTC').tz_localize(None)
    fig_bt.add_trace(go.Scatter(x=eq_index, y=equity_curve, mode='lines', name='Equity Curve'), row=2, col=1)
else:
    fig_bt.add_trace(go.Scatter(x=[price_index.min()], y=[initial_capital], mode='lines', name='Equity Curve'), row=2, col=1)

fig_bt.update_yaxes(title_text='Price', row=1, col=1)
fig_bt.update_yaxes(title_text='Equity', row=2, col=1)
fig_bt.update_layout(title='Interactive backtest review', template='plotly_white', xaxis_rangeslider_visible=False)
fig_bt.show()
